<a href="https://colab.research.google.com/github/kkruglik/female_streets/blob/main/russian_streets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Обработка данных

In [ ]:
pip install pymorphy2 pymorphy2-dicts DAWG-Python pandarallel -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 108.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/Adress_klassificator_FIAS+Pochta_RF_179_14.10.21.zip" -d .

Archive:  /content/drive/MyDrive/Colab Notebooks/Adress_klassificator_FIAS+Pochta_RF_179_14.10.21.zip
  inflating: ./administrative_addresses.csv  
  inflating: ./meta.csv              
  inflating: ./structure.csv         


In [ ]:
import requests
import json
import pandas as pd
import pymorphy2
from pandarallel import pandarallel
import numpy as np

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
pd.set_option('display.max_rows', 200)

In [ ]:
male_names = [
    'август', 'агап', 'агафон', 'адам', 'адриан', 'азарий', 'аким', 'алан', 'александр', 'алексей',
    'альберт', 'анатолий', 'андрей', 'антип', 'антон', 'анфим', 'аполлинарий', 'арий', 'аристарх',
    'аркадий', 'арно', 'арнольд', 'арсений', 'артем', 'артемий', 'артур', 'архип', 'афанасий',
    'боголюб', 'болеслав', 'борис', 'борислав', 'бронислав', 'вадим', 'валентин', 'валерий',
    'вальдемар', 'варфоломей', 'василий', 'ватслав', 'велизар', 'венедикт', 'вениамин', 'викентий',
    'виктор', 'вилен', 'виссарион', 'виталий', 'владимир', 'владислав', 'владлен', 'власий', 'володар',
    'всеволод', 'вячеслав', 'гектор', 'геннадий', 'георгий', 'герасим', 'герман', 'глеб', 'горислав',
    'григорий', 'даниил', 'данислав', 'демид', 'демьян', 'денис', 'дмитрий', 'добрыня', 'дорофей',
    'евгений', 'евграф', 'евдоким', 'евсей', 'евстафий', 'егор', 'елизар', 'елисей', 'емельян',
    'еремей', 'ермак', 'ермил', 'ермолай', 'ерофей', 'ефим', 'ефрем', 'жан', 'захар', 'зигмунд',
    'зиновий', 'зосима', 'зураб', 'иван', 'игнат', 'игнатий', 'игорь', 'илларион', 'илья', 'инокентий',
    'ипполит', 'казимир', 'кай', 'камиль', 'ким', 'кир', 'кирилл', 'клавдий', 'клим', 'кондрат',
    'константин', 'корней', 'корнелий', 'кузьма', 'лавр', 'лаврентий', 'лазарь', 'ларион', 'лев',
    'леонард', 'леонид', 'леонтий', 'лука', 'лукьян', 'любомир', 'макар', 'макарий', 'максим',
    'максимильян', 'марк', 'маркел', 'марсель', 'мартин', 'матвей', 'мечеслав', 'микула', 'милан',
    'мирон', 'мирослав', 'митрофан', 'михаил', 'модест', 'мстислав', 'нестор', 'никанор', 'никита',
    'никифор', 'никодим', 'никола', 'николай', 'олег', 'онисим', 'орест', 'осип', 'остап', 'павел',
    'панкрат', 'парамон', 'пахом', 'петр', 'порфирий', 'потап', 'прокофий', 'прохор', 'радомир',
    'радослав', 'рафаэль', 'ренат', 'роберт', 'родион', 'ролан', 'роман', 'ростислав', 'рудольф',
    'савелий', 'светозар', 'святослав', 'семен', 'серафим', 'сергей', 'спартак', 'спиридон',
    'станислав', 'степан', 'стефан', 'тарас', 'терентий', 'тимофей', 'тихомир', 'тихон', 'трифон',
    'трофим', 'ульян', 'устин', 'федор', 'федот', 'феликс', 'феофан', 'филарет', 'филат', 'филимон',
    'филипп', 'фирс', 'фома', 'фрол', 'харитон', 'эдуард', 'эмилий', 'эраст', 'эрик', 'эрнест', 'юлиан',
    'юлий', 'юрий', 'юст', 'яков', 'ян', 'януарий', 'яромир', 'ярослав'
                    ]

female_names = [
    'августа', 'авдотья', 'аврора', 'агата', 'аглая', 'агнесса', 'агния', 'ада', 'адель', 'аза',
    'азалия', 'аида', 'аксинья', 'акулина', 'алевтина', 'александра', 'алексия', 'алёна', 'алена', 'алина',
    'алиса', 'алла', 'альберта', 'амалия', 'анастасия', 'ангелина', 'андриана', 'анеля', 'анжела',
    'анжелика', 'анисья', 'анита', 'анна', 'антонина', 'анфиса', 'апрелия', 'ариадна', 'арина',
    'арсения', 'ассоль', 'астра', 'ася', 'аурика', 'аэлита', 'бажена', 'беата', 'беатриса', 'белла',
    'беляна', 'береслава', 'берта', 'богдана', 'божена', 'бронислава', 'валентина', 'валерия',
    'ванда', 'ванесса', 'варвара', 'василина', 'василиса', 'васса', 'веда', 'венера', 'вера',
    'вероника', 'веселина', 'веста', 'вета', 'вивиана', 'видана', 'виктория', 'вилора', 'виола',
    'виринея', 'виталия', 'влада', 'владилена', 'владимира', 'владислава', 'власта', 'воля',
    'всеслава', 'габриэлла', 'гайя', 'галина', 'гелия', 'гелла', 'гера', 'герда', 'глафира', 'глория',
    'грация', 'грета', 'дайна', 'дана', 'даниэла', 'дарина', 'дарья', 'дарьяна', 'декабрина', 'дея',
    'дженни', 'джулия', 'диана', 'дина', 'добрава', 'доля', 'доминика', 'дорофея', 'ева', 'евгения',
    'евдокия', 'екатерина', 'елена', 'елизавета', 'емельяна', 'есения', 'ефимия', 'ефросиния',
    'жанна', 'жасмин', 'ждана', 'забава', 'залина', 'зара', 'зарема', 'зарина', 'звана', 'звенислава',
    'земфира', 'зинаида', 'зиновия', 'зита', 'злата', 'златослава', 'зоряна', 'зоя', 'иванна',
    'ивона', 'ида', 'изабелла', 'изольда', 'илона', 'иля', 'инга', 'инесса', 'инна', 'иоанна',
    'иоланта', 'ирина', 'ирма', 'искра', 'июлия', 'камилла', 'карина', 'каролина', 'катарина', 'кира',
    'клавдия', 'клара', 'кристина', 'ксения', 'лада', 'лана', 'лара', 'лариса', 'лаура', 'леля',
    'лера', 'леся', 'лиана', 'лидия', 'лика', 'лилиана', 'лилия', 'лина', 'линда', 'лира', 'лия',
    'лолита', 'лора', 'луиза', 'любава', 'любовь', 'людмила', 'майя', 'малика', 'мальвина',
    'маргарита', 'марианна', 'марина', 'мария', 'марта', 'матрена', 'мелания', 'мелисса', 'мила',
    'милада', 'милана', 'милолика', 'милослава', 'мира', 'мирослава', 'мишель', 'мия', 'млада',
    'моника', 'муза', 'надежда', 'нана', 'наталья', 'нева', 'нелли', 'ника', 'николина', 'николь',
    'нила', 'нина', 'нинель', 'нона', 'оксана', 'олеся', 'олимпия', 'ольга', 'пелагея', 'полина',
    'прасковья', 'рада', 'раиса', 'регина', 'рената', 'риана', 'римма', 'рита', 'роберта', 'рогнеда',
    'роза', 'роксана', 'ростислава', 'рузана', 'руслана', 'руфина', 'сабрина', 'сандра', 'светлана',
    'святослава', 'северина', 'селена', 'серафима', 'сильвия', 'слава', 'славяна', 'снежана', 'соня',
    'софия', 'станислава', 'стелла', 'стефания', 'таисия', 'тамара', 'татьяна', 'ульяна', 'урсула',
    'услада', 'устина', 'фаина', 'феба', 'фекла', 'фрида', 'хана', 'хельга', 'хлоя', 'христина',
    'царина', 'цветана', 'цецилия', 'чулпан', 'шанель', 'шарлотта', 'шейла', 'эвелина', 'эдита',
    'элеонора', 'элина', 'элла', 'элоиза', 'эльвира', 'эльза', 'эльмира', 'эмилия', 'эмма', 'эрика',
    'эсмеральда', 'эстер', 'эшли', 'юлиана', 'юлия', 'юна', 'юния', 'юнона', 'ядвига', 'яна', 'янина',
    'ярина', 'яромира', 'ярослава', 'ясмина', 'фатиме'
                      ]

In [ ]:
# кол-во уникальных женских имен
len(set(female_names))

309

In [ ]:
# кол-во уникальных мужских имен
len(set(male_names))

220

In [ ]:
def get_gender(name):
    count = {'Female':0, 'Male':0, 'Unk':0}
    name_tokens = name.split(' ')

    # переводим токены в именительный падеж
    norm_names = []
    for n in name_tokens:
        n = morph.parse(n)[0]
        n = n.normal_form
        norm_names.append(n)

    # проверяем есть ли токены в словарях с муж. и жен. именами
    for n in norm_names:
        n = n.lower().strip().replace('ё', 'е')
        if n in female_names:
            count['Female'] += 1

        if n in male_names:
            count['Male'] += 1

    if count["Female"] > count["Male"]:
        return "Female"
    elif count["Female"] < count["Male"]:
        return "Male"
    return "Unk"

In [ ]:
# female_names = pd.read_csv(
#     "https://raw.githubusercontent.com/sorokinpf/russian_names/master/russian_female_names.csv",
#     names=['name', 'count'],
#     header=None)


# male_names = pd.read_csv(
#     "https://raw.githubusercontent.com/sorokinpf/russian_names/master/russian_male_names.csv",
#     names=['name', 'count'],
#     header=None)


# male_names = male_names.name.to_list()
# female_names = female_names.name.to_list()

In [ ]:
import re

def remove_special_characters(string):
    # Define the pattern to match special characters
    pattern = r'[^\w\sа-яА-Я]'

    # Remove special characters using regular expressions
    string = re.sub(pattern, '', string)

    return string

def remove_digit_letter(string):
    # Pattern to match digits followed by letters
    pattern = r'\d+\p{L}'
    # Remove digit-letter combinations using regex
    cleaned_string = re.sub(pattern, '', string)
    return cleaned_string

In [ ]:
def get_gender(name):
    count = {'Female':0, 'Male':0, 'Unk':0}
    name_tokens = name.split(' ')

    # переводим токены в именительный падеж
    norm_names = []
    for n in name_tokens:
        n = morph.parse(n)[0]
        n = n.normal_form
        norm_names.append(n)

    # проверяем есть ли токены в словарях с муж. и жен. именами
    for n in norm_names:
        n = n.lower().strip().replace('ё', 'е')
        if n in female_names:
            count['Female'] += 1

        if n in male_names:
            count['Male'] += 1

    if count["Female"] > count["Male"]:
        return "Female"
    elif count["Female"] < count["Male"]:
        return "Male"
    return "Unk"

In [ ]:
# загрузка датасета
dataset = pd.read_csv("administrative_addresses.csv", dtype="str", sep=";")

In [ ]:
df = dataset[dataset['street'].notnull()].copy()

In [ ]:
# убираем ненужные признаки
df = df.filter(items=['region', 'settlement', 'street', 'street_type'])

In [ ]:
df.street = df.street.str.lower()
df.street = df.street.apply(remove_special_characters)
df.street = df.street.str.replace('имени', '', regex=False)
df.street = df.street.str.replace('им ', '', regex=False)
# df.street = df.street.apply(remove_digit_letter)
df.street = df.street.str.replace('\d+', '', regex=True)
df.street = df.street.str.strip()

In [ ]:
# оставляем только уникальные улицы для каждого населенного пункта в России
df = df.drop_duplicates(subset=['region', 'settlement', 'street', 'street_type'])
df.shape

(834184, 4)

In [ ]:
df.street = df.street.str.replace('й ', '', regex=False)
df.street = df.street.str.replace('я ', '', regex=False)
df.street = df.street.str.replace(' й', '', regex=False)

In [ ]:
df.loc[df.street == "зои космодемьянской й", "street"] = "зои космодемьянской"
df.loc[df.street == "й зои космодемьянской", "street"] = "зои космодемьянской"
df.loc[df.street == "й розы люксембург", "street"] = "розы люксембург"


Сколько уникальных улиц в каждом регионе

In [ ]:
dataset.groupby("region").size().to_frame()

,0
region,
Адыгея,157168
Алтай,78231
Алтайский,466001
Амурская,146895
Архангельская,181557
Астраханская,246314
Байконур,429
Башкортостан,710773
Белгородская,462829


In [ ]:
# определяем мужская или женская улица
df["genders"] = df.street.parallel_apply(lambda x: get_gender(x))

In [ ]:
df = df[df.genders != "Unk"]

In [ ]:
# df.street = df.street.str.lower()
# df.street = df.street.str.strip()
# df.street = df.street.str.replace('\d+', '')

## 2. Отчистка

В женские улицы попадают нерелевантные названия. Например, улица Надежды, Веры, Любви и так далее. С одной стороны это действительно женское имя, но названы эти улицы не честь женщин.

In [ ]:
remove_names = [
    "надежды",
    "славы",
    "роз",
    "искра",
    "надежда",
    "воинской славы",
    "чулпан",
    "алых роз",
    "народной воли",
    "трудовой славы",
    "веры",
    "любви",
    "шахтерской славы",
    "материнской славы",
    "светлых надежд",
    "нева",
    "искры",
    "надежд",
    "красных роз",
    "боевой славы",
    "сдт виктория",
    "аврора",
    "лилий",
    "виктория",
    "виктории",
    "воли",
    "воля",
    "снт надежда",
    "ратной славы",
    "роза",
    "роза ветров",
    "бульвар роз",
    "ким",
    "кима",
    "5 августа",
    "аллея спортивной славы",
    "славы 1-й",
    "славы -й",
    "славы",
    "белых роз",
    "газеты искра",
    "николина",
    "панаева ид",
    "венера",
    "янина",
    "красная линда",
    "линда",
    "розы",
    "я надежда",
    "долина роз",
    "белых роз",
    "ид иванова",
    "набережная р невы",
    "доброй надежды",
    "ариадны",
    "сдт надежда",
    "сопка любви",
    "софия",
    "слава киз",
    "ангелиной",
    "августа"
]

In [ ]:
male_streets = [
    "валентина эмирова",
    "валентина берестова",
    "гере гаджитаев",
    "малика джаналиева"
]

In [ ]:
df.loc[df.street.isin(remove_names), "genders"] = "Unk"

<ipython-input-27-b7289db0dacb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.street.isin(remove_names), "genders"] = "Unk"


In [ ]:
df.loc[df.street.isin(male_streets), "genders"] = "Male"

<ipython-input-28-f890fe9bb44d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.street.isin(male_streets), "genders"] = "Male"


In [ ]:
df = df[df.genders != "Unk"]

## 3. Исследование

**Какой датасет мы анализируем**

Набор содержит объединённые из двух источников (ФИАС и обработанные порталом kodifikant.ru данные Почты России) географические данные РФ с детализацией до дома.

Цель датасета — создание единого максимально полного классификатора географических данных, с помощью которого можно было бы осуществлять поиск и привязку географических объектов к официальным стандартизированным геоданным. Для осуществления этой привязки можно использовать библиотеку, доступную по ссылке.

Единица наблюдения в датасете — адрес объекта недвижимости. Данные для каждого наблюдения приведены по следующим атрибутам: федеральный округ, субъект РФ, муниципальное образование, населенный пункт, локация, улица, дом.

Набор данных охватывает все регионы Российской Федерации. Всего набор содержит **25 569 964 наблюдений** по 11 атрибутам.

[Источник и описание](https://www.data-in.ru/data-catalog/datasets/179/)

В изначальном датасете больше 20 млн строк. Такое количество данных получается из-за высокой детализации данных, вплоть до дома. Нам для анализа такая детализация не нужна.

Поэтому оставим только уникальные названия улиц для каждого населённого пункта России. Таким образом у нас остаётся порядка 800 тысяч значений, в которых мы и будем искать женские и мужские улицы.

Пример нашего датасета:

In [ ]:
df.sample(10)

,region,settlement,street,street_type,genders
16619013,Татарстан,Елабуга,зои космодемьянской,заезд,Female
2194005,Пермский,Чусовой,максима горького,улица,Male
15584870,Башкортостан,Райманово,ивана елкина,переулок,Male
3660917,Брянская,Залядка,бориса вескова,улица,Male
2244968,Пермский,Пашия,максима горького,улица,Male
7788659,Новосибирская,Барабинск,льва толстого,переулок,Male
24351829,Тверская,Красномайский,максима горького,улица,Male
11363548,Тюменская,Тюмень,семёна урусова,улица,Male
4971387,Иркутская,Мамоны,александра дербышева,улица,Male
23711148,Саратовская,Горный,степана разина,улица,Male


Всего в нашем датасете получилось **1490 женских** и **7598 мужских** улицы. Мужских улиц почти в 4 раза больше.

И наверное их было бы ещё больше. Но мужские и женские улицы мы ищем по наличию имён в названиях. Соотвественно улица Ленина в топ мужских улиц не попадает, потому что в ней нет самого имени.

In [ ]:
# сколько мужских и женских улиц
df.groupby('genders').size().to_frame()

,0
genders,
Female,1514
Male,5618


В топе наиболее распространённых улиц без особых сюрпризов. Самая часто встречающая женская улица названы в честь Розы Люксембург и Клары Цеткин - революционерок с именами которых связан 8 марта. А также улицы в честь Зои Космодемьянской - первая женщина, удостоенная звания Героя Советского Союза во время Великой Отечественной войны.

In [ ]:
# самая распространнённая женская улица
df[df.genders == "Female"] \
    .groupby("street").size().sort_values(ascending=False).to_frame().head(100)

,0
street,
розы люксембург,285
зои космодемьянской,262
клары цеткин,114
полины осипенко,58
марины расковой,25
марины цветаевой,24
валентины терешковой,16
анны ахматовой,16
марии ульяновой,12


Самый популярая мужчиной стал Степан Разин

In [ ]:
# самая распространнённая мужская улица
df[df.genders == "Male"] \
    .groupby("street").size().sort_values(ascending=False).head(100).to_frame()

,0
street,
степана разина,475
максима горького,436
льва толстого,336
олега кошевого,265
демьяна бедного,183
спартака,169
ермака,145
александра невского,96
александра матросова,82


Кол-во мужских и женских улиц в регионах

In [ ]:
result = df.groupby(['genders', 'region']).size().unstack('genders')
result[['Female', 'Male']].sort_values('Female', ascending=False).head(100)

genders,Female,Male
region,,
Краснодарский,114.0,298.0
Свердловская,102.0,310.0
Пермский,63.0,146.0
Челябинская,61.0,170.0
Ростовская,48.0,162.0
Московская,47.0,152.0
Татарстан,42.0,193.0
Крым,39.0,102.0
Брянская,37.0,64.0


И города

In [ ]:
result = df.groupby(['genders', 'settlement']).size().unstack('genders')
result[['Female', 'Male']].sort_values('Female', ascending=False).head(200)

genders,Female,Male
settlement,,
Новороссийск,19.0,25.0
Волгоград,14.0,34.0
Краснодар,13.0,75.0
Тверь,12.0,21.0
Керчь,11.0,17.0
Киров,11.0,18.0
Тюмень,10.0,69.0
Астрахань,10.0,37.0
Воронеж,9.0,26.0


Самый профеминисткий город в плане названий стал Волгоград и Новроссийск.

In [ ]:
# в каком городе больше всего женских улиц
df[df.genders == "Female"] \
    .groupby(['settlement']).size().sort_values(ascending=False).head(10).to_frame()

,0
settlement,
Новороссийск,19
Волгоград,14
Краснодар,13
Тверь,12
Керчь,11
Киров,11
Тюмень,10
Астрахань,10
Воронеж,9


Что это за улицы на примере Волгограда и Новороссийска:

In [ ]:
df[(df.settlement == "Волгоград") & (df.genders == "Female")]

,region,settlement,street,street_type,genders
3942018,Волгоградская,Волгоград,розы люксембург,переулок,Female
3946491,Волгоградская,Волгоград,маргариты агашиной,улица,Female
3948921,Волгоградская,Волгоград,валентины терешковой,улица,Female
3973422,Волгоградская,Волгоград,елены колесовой,улица,Female
3973462,Волгоградская,Волгоград,елены стемпковской,улица,Female
3975664,Волгоградская,Волгоград,клары цеткин,улица,Female
3978319,Волгоградская,Волгоград,любови шевцовой,улица,Female
3978861,Волгоградская,Волгоград,марины расковой,улица,Female
3981456,Волгоградская,Волгоград,надежды крупской,улица,Female
3982475,Волгоградская,Волгоград,ольги ковалевой,улица,Female


In [ ]:
df[(df.settlement == "Новороссийск") & (df.genders == "Female")]

,region,settlement,street,street_type,genders
1184969,Краснодарский,Новороссийск,клары цеткин,улица,Female
1189822,Краснодарский,Новороссийск,евдокии бершанской,улица,Female
1193956,Краснодарский,Новороссийск,нины марухно,улица,Female
1195470,Краснодарский,Новороссийск,надежды онайко,улица,Female
1199863,Краснодарский,Новороссийск,нины друзяка,улица,Female
1199895,Краснодарский,Новороссийск,светланы летт,улица,Female
1199919,Краснодарский,Новороссийск,татьяны мороз,улица,Female
1204349,Краснодарский,Новороссийск,галины гаркуненко,улица,Female
1204388,Краснодарский,Новороссийск,дарьи прохорской,улица,Female
1204434,Краснодарский,Новороссийск,екатерины яйлоян,улица,Female


Самым мужскими городами оказались Тюмень и Краснодар:

In [ ]:
#в каком городе больше всего мужских улиц
df[df.genders == "Male"] \
    .groupby(['settlement']).size().sort_values(ascending=False).head(10).to_frame()

,0
settlement,
Краснодар,75
Тюмень,69
Бугульма,54
Бийск,53
Дагестанские Огни,40
Новосибирск,38
Казань,38
Астрахань,37
Ижевск,35


In [ ]:
df[(df.settlement == "Тюмень") & (df.genders == "Male")].head(100)

,region,settlement,street,street_type,genders
11319162,Тюменская,Тюмень,бориса щербины,бульвар,Male
11320886,Тюменская,Тюмень,семёна шахлина,переулок,Male
11321071,Тюменская,Тюмень,владимира хуторянского,площадь,Male
11322308,Тюменская,Тюмень,бориса маркова,проезд,Male
11334550,Тюменская,Тюмень,александра логунова,улица,Male
11334591,Тюменская,Тюмень,александра матросова,улица,Male
11334617,Тюменская,Тюмень,александра митинского,улица,Male
11334623,Тюменская,Тюмень,александра протозанова,улица,Male
11334643,Тюменская,Тюмень,александра пушкина,улица,Male
11334663,Тюменская,Тюмень,александра рывтикова,улица,Male


При этом если смотреть по региону, то в целом Краснодарский край лидер по количеству как женских улиц:

In [ ]:
# в каком регионе больше всего женских улиц
df[df.genders == "Female"] \
    .groupby(['region']).size().sort_values(ascending=False).head(10).to_frame()

,0
region,
Краснодарский,114
Свердловская,102
Пермский,63
Челябинская,61
Ростовская,48
Московская,47
Татарстан,42
Крым,39
Брянская,37


In [ ]:
df[(df.region == "Краснодарский") & (df.genders == "Female")] \
    .groupby(["street"]).size().sort_values(ascending=False).to_frame().head(100)

,0
street,
розы люксембург,27
клары цеткин,7
зои космодемьянской,7
полины осипенко,3
евдокии бершанской,3
светланы летт,2
надежды онайко,2
нины друзяка,2
малоземельца валентины зинько,2


Так и мужских улиц:

In [ ]:
# в каком регионе больше всего мужских улиц
df[df.genders == "Male"] \
    .groupby(['region']).size().sort_values(ascending=False).head(10).to_frame()

,0
region,
Свердловская,310
Краснодарский,298
Татарстан,193
Саха /Якутия/,176
Башкортостан,173
Челябинская,170
Ростовская,162
Дагестан,158
Московская,152


Алтай, Ингушетия и Кабардино-Балкария оказались регионами всего с одной женской улицой:

In [ ]:
df[df.genders == "Female"] \
    .groupby(['region']).size().sort_values(ascending=False).tail(10).to_frame()


,0
region,
Карелия,3
Еврейская,2
Амурская,2
Рязанская,2
Чукотский,2
Ямало-Ненецкий,2
Камчатский,2
Кабардино-Балкарская,1
Ингушетия,1


In [ ]:
least_female_cities = df[df.genders == "Female"] \
    .groupby(['region']).size().sort_values(ascending=False).tail(10).to_frame().index.tolist()

In [ ]:
# список женских улиц в регионах, где почти нет женских улиц
df[(df.region.isin(least_female_cities)) & (df.genders == "Female")] \
    .groupby(["region", "street"]).size().sort_values(ascending=False).to_frame().head(100)

0
region               street                       
Алтай                евдокии мекшило             1
Амурская             зои космодемьянской         1
Ямало-Ненецкий       анны пантелеевой            1
Чукотский            галины николаевны тагриной  1
                     анны тынетегиной            1
Рязанская            полина                      1
                     дарьи гармаш                1
Карелия              розы люксембург             1
                     анны романовой              1
                     анны лисициной              1
Камчатский           любови шевцовой             1
                     веры орловой                1
Кабардино-Балкарская марии шекихачевой           1
Ингушетия            зои космодемьянской         1
Еврейская            розы люксембург             1
                     зои космодемьянской         1
Амурская             полины осипенко             1
Ямало-Ненецкий       зои космодемьянской         1

Ну и посмотрим какая женская улица наиболее популярна в каждом регионе России. Может быть тут будут какие-то региональные особенности.

In [ ]:
grouped = df[df.genders == "Female"].groupby(["region", "street"], as_index=False).size()
idx = grouped.groupby("region")["size"].idxmax()
grouped.loc[idx]

,region,street,size
3,Адыгея,лели богузоковой,2
4,Алтай,евдокии мекшило,1
9,Алтайский,розы люксембург,6
11,Амурская,зои космодемьянской,1
21,Архангельская,розы шаниной,5
30,Астраханская,зои космодемьянской,9
45,Башкортостан,зои космодемьянской,3
63,Белгородская,зои космодемьянской,2
83,Брянская,розы люксембург,9
85,Бурятия,елены волковой,1


**Что ещё можно сделать?**
- добавить к именам классификацию. Например, поэт или советский революционер, чтобы посмотреть кто из них преобладает в названиях улиц.
- добавить жизни к имена, чтобы посмотреть к какому периоду чаще отсылают названия улиц

**Гипотезы:**
- Женские имена в названиях улиц представлены значительно меньше, чем мужские.
- Использование женских имен в названиях улиц варьируется в зависимости от региона и зависит от культурных, исторических и социальных факторов
- Женские имена с большей вероятностью будут использоваться в названиях улиц в районах с более высоким процентом женского населения.
- Женские имена чаще используются в названиях улиц в городах с большим числом женщин-политических лидеров или там, где исторически были заметны движения за права женщин.
- Женские имена в названиях улиц чаще используются для жилых улиц, чем для крупных дорог и магистралей.

In [ ]:
# df[df.settlement == "Калининград"]

In [ ]:
for i in dataset.loc[dataset.settlement == "Калининград", "street"].unique():
    print(i)

Знаний
Борисовский
Л.Шевцовой
Солнечный
Ф.Лефорта
Южный
Адмирала Трибуца
Ветеранов
Генерала Карбышева
Петра Великого
Правая
Старопрегольская
А.Дубровиной
А.Орлова
А.Толстого
Алданский
Алданский 2-й
Альпийский 2-й
Аральский
Арзамасский
Арсенальный
Бабаева
Балтийский
Батальный
Беланова
Береговой
Бодайбинский
Большевистский
Броненосный
Букетный
Венский
Воздушный
Войнова
Волоколамский
Восточный
Г.Тимофеевой 1-й
Г.Щербакова
Ганзейский
Грибоедова
Грига
Двинской
Джамбула
Достоевского
Дружный
Енисейский
Желябова
Заводской
Загородный
Знаменский
Зоологический
Иртышский
Калинина
Калужский
Камский 1-й
Камский 2-й
Карташева
Качалова
Каштановый
Киевский
Кирова
Клинской
Коммунальный
Косогорный
Краснодонский
Краснознаменский
Краснокаменный
Крылова
Куйбышева
Кутаисский
Ладушкина
Лаптевых
Левитана
Лесная аллея
Лесопарковый
Литовский
Лозовой
Ломоносова
Лукашова
Лучистый
Львовский
М.Зощенко
М.Светлова
М.Цветаевой
Майский
Малый
Минина и Пожарского
Многопрудный
Можайский
Мореходный
Нансена
Нарвский
Нахимова

## Additional research

In [ ]:
df

,region,settlement,street,street_type,genders
1066,Еврейская,Биробиджан,владимира вайсера,улица,Male
1069,Еврейская,Биробиджан,владимира пеллера,улица,Male
3605,Еврейская,Биробиджан,олега кошевого,улица,Male
25949,Еврейская,Смидович,розы люксембург,улица,Female
27769,Еврейская,Камышовка,зои космодемьянской,улица,Female
...,...,...,...,...,...
25554521,Севастополь,NaN,максима рыбальченко,улица,Male
25555455,Севастополь,Кача,бориса сафонова,улица,Male
25556351,Севастополь,NaN,михаила дзигунского,улица,Male
25557408,Севастополь,Инкерман,ивана личкатого,улица,Male


In [ ]:
add_names = ["крупской", "крупская", "люксембург", "цеткин", "космодемьянской", "космодемьянская",
             "ульяновой"
]

In [ ]:
dataset.street = dataset.street.str.lower()

In [ ]:
dataset = dataset[dataset.street.notna()]

In [ ]:
mask = dataset['street'].str.contains('|'.join(add_names))

In [ ]:
dataset[mask].groupby("street").size().sort_values(ascending=False).to_frame()

,0
street,
крупской,36668
розы люксембург,14098
зои космодемьянской,7421
р.люксембург,5077
клары цеткин,3917
з.космодемьянской,3625
крупская,3577
космодемьянской,1806
к.цеткин,1120
